In [39]:
import warnings

import numpy as np
import pandas as pd

from jre_utils.datapath import (
    factor_data_paths,
    model_ready_data_paths,
    get_derived_csv_path,
)
from jre_utils.config import asset_types

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [40]:
asset_type = "combined"
dataset_key = "transactions"
years_ahead = 2

metrics = {
    "median": "unit_price_median",
    "gmean": "unit_price_gmean",
}

granularity_columns = ["area", "area_code"]
group_by_columns = granularity_columns + ["year"]

metric_key = "gmean"
metric = metrics[metric_key]
metric_pct_chg = metric + "_pct_chg"


In [41]:
population_path = factor_data_paths["processed"]["population"]["municipality"]
migration_path = factor_data_paths["processed"]["migration"]["municipality"]
taxable_income_path = factor_data_paths["processed"]["taxable_income"]["municipality"]
new_dwellings_path = factor_data_paths["processed"]["new_dwellings"]["municipality"]
lfs_revenue_path = factor_data_paths["processed"]["lfs_revenue_breakdown"]["municipality"]

dataset_name = f"sequence_{dataset_key}_{asset_type}_{metric_key}_{years_ahead}"
model_ready_data_path = model_ready_data_paths[dataset_name]

In [42]:
# concal all asset types horizontally here
# Once creating time series dataset,
# create 3 datasets and concat them vertically
# To do that, loop 3 times with different metrics

In [43]:
id_columns = ["area_code", "area", "year"]

factor_log_normalize_columns = [
    "in_migrations",
    "out_migrations",
    "population",
    "taxpayer_count",
    "taxable_income",
    "taxable_income_per_taxpayer",
    "total_tax",
    "new_dwellings",
    "existing_dwellings",
]

factor_normalize_columns = [
    "total_tax_growth",
    "taxable_income_growth",
    "taxable_income_per_taxpayer_growth",
    "net_migration_ratio",
    "new_dwellings_ratio",
    "taxpayer_count_growth",
]

factor_maintain_columns = [
    "migrations_is_available",
    "taxable_income_is_available",
    "dwellings_is_available",
    "total_tax_is_available",
]

factor_columns = (
    [f"{column}_log_normalized_yearly" for column in factor_log_normalize_columns]
    + [f"{column}_normalized_yearly" for column in factor_normalize_columns]
    + factor_maintain_columns
)

final_factor_columns = factor_normalize_columns + factor_log_normalize_columns + factor_columns

In [44]:
core_log_normalize_columns = ["count", metric]
core_normalize_columns = ["count_growth", "yearly_price_growth", metric_pct_chg]
core_maintain_columns = ["metric_pct_chg_is_available"]

core_columns = (
    [f"{column}_log_normalized_yearly" for column in core_log_normalize_columns]
    + [f"{column}_normalized_yearly" for column in core_normalize_columns]
    + core_maintain_columns
)

final_core_columns = core_normalize_columns + core_log_normalize_columns + core_columns

combined_final_core_columns = [f"{asset_type}_{column}" for column in final_core_columns for asset_type in asset_types]

In [45]:
final_columns = id_columns + combined_final_core_columns + final_factor_columns

In [46]:
derived_dfs = {}

for asset_type in asset_types:
    df =  pd.read_csv(get_derived_csv_path(asset_type))
    df = df.sort_values(by=group_by_columns, ascending=True)
    df = df[group_by_columns + [metric, "count"]]

    # prepare main metric
    df[metric_pct_chg] = df.groupby(granularity_columns)[metric].pct_change(periods=years_ahead)

    # prepare additional factors
    df["count_growth"] = df.groupby(granularity_columns)["count"].pct_change()
    df["yearly_price_growth"] = df.groupby(granularity_columns)[metric].pct_change()
    df["metric_pct_chg_is_available"] = df[metric_pct_chg].notnull().astype(int)

    for column in ["count", metric]:
        df[f"{column}_log"] = df[column].apply(lambda x: np.log10(1 + x))
        df[f"{column}_log_normalized_yearly"] = df.groupby("year")[f"{column}_log"].transform(
            lambda x: (x - x.mean()) / x.std()
        )

    for column in ["count_growth", "yearly_price_growth", metric_pct_chg]:
        df[f"{column}_normalized_yearly"] = df.groupby("year")[column].transform(
            lambda x: (x - x.mean()) / x.std()
        )

    df = df[group_by_columns + final_core_columns]
    
    derived_dfs[asset_type] = df.rename(
        columns={column: f"{asset_type}_{column}" for column in final_core_columns}
    )

combined_derived_dfs = (
    derived_dfs["building"].merge(
        derived_dfs["land"],
        on=["year", "area_code", "area"],
        how="outer",
    ).merge(
        derived_dfs["condo"],
        on=["year", "area_code", "area"],
        how="outer",
    )
)

In [47]:
combined_derived_dfs

,area,area_code,year,building_count_growth,building_yearly_price_growth,building_unit_price_gmean_pct_chg,building_count,building_unit_price_gmean,building_count_log_normalized_yearly,building_unit_price_gmean_log_normalized_yearly,building_count_growth_normalized_yearly,building_yearly_price_growth_normalized_yearly,building_unit_price_gmean_pct_chg_normalized_yearly,building_metric_pct_chg_is_available,land_count_growth,land_yearly_price_growth,land_unit_price_gmean_pct_chg,land_count,land_unit_price_gmean,land_count_log_normalized_yearly,land_unit_price_gmean_log_normalized_yearly,land_count_growth_normalized_yearly,land_yearly_price_growth_normalized_yearly,land_unit_price_gmean_pct_chg_normalized_yearly,land_metric_pct_chg_is_available,condo_count_growth,condo_yearly_price_growth,condo_unit_price_gmean_pct_chg,condo_count,condo_unit_price_gmean,condo_count_log_normalized_yearly,condo_unit_price_gmean_log_normalized_yearly,condo_count_growth_normalized_yearly,condo_yearly_price_growth_normalized_yearly,condo_unit_price_gmean_pct_chg_normalized_yearly,condo_metric_pct_chg_is_available
0,Aichi-ken Agui-cho,23441,2007,NaN,NaN,NaN,6.0,67590.233575,-1.205808,0.066006,NaN,NaN,NaN,0.0,NaN,NaN,NaN,20.0,54723.779572,-0.544187,0.674598,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aichi-ken Agui-cho,23441,2008,1.333333,1.105567,NaN,14.0,142315.742351,-0.674282,0.968399,0.497956,3.158972,NaN,0.0,0.500000,-0.250691,NaN,30.0,41004.994789,-0.290165,0.511063,0.118376,-0.581869,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aichi-ken Agui-cho,23441,2009,1.071429,0.024848,1.157887,29.0,145852.056278,-0.183325,1.103548,1.470449,0.151198,3.510887,1.0,0.233333,0.473600,0.104181,37.0,60424.944041,-0.023743,0.964866,0.479061,1.331892,0.359541,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aichi-ken Agui-cho,23441,2010,0.413793,-0.019251,0.005119,41.0,143044.196187,0.055760,1.082660,0.690053,-0.133281,0.093175,1.0,0.459459,0.044494,0.539166,54.0,63113.475459,0.231435,1.015879,0.515716,-0.019356,1.443527,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aichi-ken Agui-cho,23441,2011,-0.024390,0.010080,-0.009365,40.0,144486.087473,0.100640,1.102463,0.004318,-0.085690,-0.082755,1.0,-0.203704,0.098042,0.146898,43.0,69301.255966,0.057424,1.110362,-0.533401,0.159122,0.204269,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21230,Yamanashi-ken Oshino-mura,19424,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.181818,1.101751,1.366668,9.0,23205.212513,-1.285960,0.237574,-0.415074,1.490448,1.809799,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21231,Yamanashi-ken Oshino-mura,19424,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.333333,-0.214747,0.650406,12.0,18221.953987,-1.091118,0.046115,0.438089,-0.585584,0.756029,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21232,Yamanashi-ken Oshino-mura,19424,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.250000,0.612252,0.266025,9.0,29378.375210,-1.340108,0.418723,-0.791044,0.929838,0.299287,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21233,Yamanashi-ken Oshino-mura,19424,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.111111,-0.497405,-0.189691,8.0,14765.418036,-1.345537,-0.144001,-0.132958,-1.211575,-0.549031,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
df = combined_derived_dfs

population_df = pd.read_csv(population_path)
migration_df = pd.read_csv(migration_path)
taxable_income_df = pd.read_csv(taxable_income_path)
new_dwellings_df = pd.read_csv(new_dwellings_path)
lfs_revenue_df= pd.read_csv(lfs_revenue_path)

df = (
    df.merge(population_df, on=group_by_columns, how="left")
    .merge(migration_df, on=group_by_columns, how="left")
    .merge(taxable_income_df, on=group_by_columns, how="left")
    .merge(new_dwellings_df, on=group_by_columns, how="left")
    .merge(lfs_revenue_df, on=group_by_columns, how="left")
)

In [49]:
df["migrations_is_available"] = df["net_migration_ratio"].notnull().astype(int)
df["taxable_income_is_available"] = df["taxable_income"].notnull().astype(int)
df["total_tax_is_available"] = df["total_tax"].notnull().astype(int)
df["dwellings_is_available"] = df["new_dwellings"].notnull().astype(int)

In [50]:
df = df[final_columns]

In [52]:
df.to_csv(model_ready_data_path, index=False)

In [53]:
df.describe()

,area_code,year,land_count_growth,building_count_growth,condo_count_growth,land_yearly_price_growth,building_yearly_price_growth,condo_yearly_price_growth,land_unit_price_gmean_pct_chg,building_unit_price_gmean_pct_chg,condo_unit_price_gmean_pct_chg,land_count,building_count,condo_count,land_unit_price_gmean,building_unit_price_gmean,condo_unit_price_gmean,land_count_log_normalized_yearly,building_count_log_normalized_yearly,condo_count_log_normalized_yearly,land_unit_price_gmean_log_normalized_yearly,building_unit_price_gmean_log_normalized_yearly,condo_unit_price_gmean_log_normalized_yearly,land_count_growth_normalized_yearly,building_count_growth_normalized_yearly,condo_count_growth_normalized_yearly,land_yearly_price_growth_normalized_yearly,building_yearly_price_growth_normalized_yearly,condo_yearly_price_growth_normalized_yearly,land_unit_price_gmean_pct_chg_normalized_yearly,building_unit_price_gmean_pct_chg_normalized_yearly,condo_unit_price_gmean_pct_chg_normalized_yearly,land_metric_pct_chg_is_available,building_metric_pct_chg_is_available,condo_metric_pct_chg_is_available,total_tax_growth,taxable_income_growth,taxable_income_per_taxpayer_growth,net_migration_ratio,new_dwellings_ratio,taxpayer_count_growth,in_migrations,out_migrations,population,taxpayer_count,taxable_income,taxable_income_per_taxpayer,total_tax,new_dwellings,existing_dwellings,in_migrations_log_normalized_yearly,out_migrations_log_normalized_yearly,population_log_normalized_yearly,taxpayer_count_log_normalized_yearly,taxable_income_log_normalized_yearly,taxable_income_per_taxpayer_log_normalized_yearly,total_tax_log_normalized_yearly,new_dwellings_log_normalized_yearly,existing_dwellings_log_normalized_yearly,total_tax_growth_normalized_yearly,taxable_income_growth_normalized_yearly,taxable_income_per_taxpayer_growth_normalized_yearly,net_migration_ratio_normalized_yearly,new_dwellings_ratio_normalized_yearly,taxpayer_count_growth_normalized_yearly,migrations_is_available,taxable_income_is_available,dwellings_is_available,total_tax_is_available
count,21235.000000,21235.000000,19393.000000,17342.000000,4911.000000,19393.000000,17342.000000,4911.000000,18100.000000,16179.000000,4576.000000,20721.000000,18542.000000,5262.000000,2.072100e+04,1.854200e+04,5.262000e+03,2.072100e+04,1.854200e+04,5.262000e+03,2.072100e+04,1.854200e+04,5.262000e+03,1.939300e+04,1.734200e+04,4.911000e+03,19393.000000,1.734200e+04,4.911000e+03,1.810000e+04,1.617900e+04,4.576000e+03,20721.000000,18542.000000,5262.000000,21141.000000,19917.000000,19917.000000,21234.000000,12175.000000,19917.000000,21234.000000,21234.000000,2.123400e+04,1.991700e+04,1.991700e+04,19917.000000,2.116400e+04,12175.000000,1.217500e+04,21234.000000,21234.000000,21234.000000,19917.000000,19917.000000,19917.000000,21164.000000,12175.000000,12175.000000,21141.000000,19917.000000,19917.000000,21234.000000,12175.000000,19917.000000,21235.000000,21235.000000,21235.000000,21235.000000
mean,21783.687921,2014.446762,0.170641,0.166097,0.106521,0.051633,0.048543,0.026340,0.039459,0.036415,0.040473,79.942908,90.882483,152.278791,4.625797e+04,9.127577e+04,2.492731e+05,-2.853007e-16,-1.870051e-16,-3.024735e-16,-2.743276e-17,3.801415e-16,-2.592630e-16,-2.931131e-18,-4.916684e-18,-2.893678e-18,0.000000,3.277789e-18,4.340518e-18,3.140520e-18,8.783519e-19,-7.763797e-19,0.873510,0.872560,0.869631,0.007955,0.003273,0.001302,-0.002976,0.013678,0.001865,3933.123293,3960.423098,9.718990e+04,4.363423e+04,1.468131e+08,2929.987478,1.490116e+07,1116.890021,6.847253e+04,0.248806,0.251034,0.291269,0.404418,0.400100,0.180698,0.539913,-0.108651,-0.102270,0.061775,0.034121,-0.012359,0.099158,-0.082956,0.092680,0.999953,0.937933,0.573346,0.996656
std,13528.582464,4.654486,2.469170,3.290677,0.448420,0.505666,0.539019,0.171165,0.519445,0.447017,0.184228,134.961584,205.934780,347.796617,1.233829e+05,1.792223e+05,1.699395e+05,9.995897e-01,9.995415e-01,9.983830e-01,9.995897e-01,9.995415e-01,9.983830e-01,9.995874e-01,9.995386e-01,9.983693e-

In [54]:
df[df[f"building_yearly_price_growth"] > 20]

,area_code,area,year,land_count_growth,building_count_growth,condo_count_growth,land_yearly_price_growth,building_yearly_price_growth,condo_yearly_price_growth,land_unit_price_gmean_pct_chg,building_unit_price_gmean_pct_chg,condo_unit_price_gmean_pct_chg,land_count,building_count,condo_count,land_unit_price_gmean,building_unit_price_gmean,condo_unit_price_gmean,land_count_log_normalized_yearly,building_count_log_normalized_yearly,condo_count_log_normalized_yearly,land_unit_price_gmean_log_normalized_yearly,building_unit_price_gmean_log_normalized_yearly,condo_unit_price_gmean_log_normalized_yearly,land_count_growth_normalized_yearly,building_count_growth_normalized_yearly,condo_count_growth_normalized_yearly,land_yearly_price_growth_normalized_yearly,building_yearly_price_growth_normalized_yearly,condo_yearly_price_growth_normalized_yearly,land_unit_price_gmean_pct_chg_normalized_yearly,building_unit_price_gmean_pct_chg_normalized_yearly,condo_unit_price_gmean_pct_chg_normalized_yearly,land_metric_pct_chg_is_available,building_metric_pct_chg_is_available,condo_metric_pct_chg_is_available,total_tax_growth,taxable_income_growth,taxable_income_per_taxpayer_growth,net_migration_ratio,new_dwellings_ratio,taxpayer_count_growth,in_migrations,out_migrations,population,taxpayer_count,taxable_income,taxable_income_per_taxpayer,total_tax,new_dwellings,existing_dwellings,in_migrations_log_normalized_yearly,out_migrations_log_normalized_yearly,population_log_normalized_yearly,taxpayer_count_log_normalized_yearly,taxable_income_log_normalized_yearly,taxable_income_per_taxpayer_log_normalized_yearly,total_tax_log_normalized_yearly,new_dwellings_log_normalized_yearly,existing_dwellings_log_normalized_yearly,total_tax_growth_normalized_yearly,taxable_income_growth_normalized_yearly,taxable_income_per_taxpayer_growth_normalized_yearly,net_migration_ratio_normalized_yearly,new_dwellings_ratio_normalized_yearly,taxpayer_count_growth_normalized_yearly,migrations_is_available,taxable_income_is_available,dwellings_is_available,total_tax_is_available
17884,6428,Yamagata-ken Shonai-machi,2018,0.166667,0.333333,NaN,-0.096588,42.673283,NaN,-0.164031,4.068458,NaN,14.0,4.0,NaN,8245.863439,46514.838675,NaN,-0.95639,-1.771965,NaN,-0.619055,0.004066,NaN,0.228354,0.785975,NaN,-0.181303,32.058345,NaN,-0.420088,8.021277,NaN,1.0,1.0,NaN,-0.000087,0.012188,0.00427,-0.007129,NaN,0.007885,407.0,553.0,20480.0,9587.0,22590577.0,2356.37603,1987466.0,NaN,NaN,-0.437891,-0.357076,-0.172928,-0.0237,-0.136917,-1.197839,-0.258908,NaN,NaN,-0.110431,-0.109632,-0.176539,-0.422169,NaN,0.116403,1,1,0,1


In [55]:
area_code = 6428
df[df["area_code"] == area_code][["year"] + [f"{asset_type}_yearly_price_growth" for asset_type in asset_types] + [f"{asset_type}_count" for asset_type in asset_types]]

,year,land_yearly_price_growth,building_yearly_price_growth,condo_yearly_price_growth,land_count,building_count,condo_count
17874,2008,-0.148374,NaN,NaN,13.0,12.0,NaN
17875,2009,-0.263423,0.021085,NaN,12.0,14.0,NaN
17876,2010,0.321346,1.464304,NaN,9.0,5.0,NaN
17877,2011,0.113980,-0.349555,NaN,12.0,10.0,NaN
17878,2012,0.062973,1.106908,NaN,17.0,5.0,NaN
17879,2013,-0.132984,-0.630526,NaN,17.0,13.0,NaN
17880,2014,-0.072015,-0.760203,NaN,13.0,4.0,NaN
17881,2015,0.145216,3.453824,NaN,13.0,10.0,NaN
17882,2016,-0.124589,-0.502936,NaN,14.0,4.0,NaN
17883,2017,-0.074654,-0.883946,NaN,12.0,3.0,NaN
